<a href="https://colab.research.google.com/github/cedro3/mediapipe/blob/main/mediapipe_for_movie.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title **set up**

# install mediapipe
!pip install mediapipe

# clone github code
!git clone https://github.com/cedro3/mediapipe.git
%cd mediapipe/

# initial setting
import mediapipe as mp
mp_holistic = mp.solutions.holistic

# Initialize MediaPipe Holistic.
holistic = mp_holistic.Holistic(
    static_image_mode=True, min_detection_confidence=0.5)

# Prepare DrawingSpec for drawing the face landmarks later.
WHITE_COLOR = (224, 224, 224)
BLACK_COLOR = (0, 0, 0)
RED_COLOR = (0, 0, 255)
GREEN_COLOR = (0, 128, 0)
BLUE_COLOR = (255, 0, 0)
mp_drawing = mp.solutions.drawing_utils 
drawing_face_spec = mp_drawing.DrawingSpec(color=WHITE_COLOR, thickness=1, circle_radius=1)
drawing_pose_spec = mp_drawing.DrawingSpec(color=WHITE_COLOR, thickness=3, circle_radius=3)
drawing_hand_spec = mp_drawing.DrawingSpec(color=WHITE_COLOR, thickness=3, circle_radius=3)
drawing_dot_spec = mp_drawing.DrawingSpec(color=RED_COLOR, thickness=2, circle_radius=3)

# define fuction
from function import *

In [ ]:
#@title **video-to-frames**
#@markdown upload video(mp4) with sound to movie/video folder
 
video = 'satomi.mp4' #@param {type:"string"}
video_file = 'video/'+video
image_dir='frames/'
image_file='%s.jpg'
 
# video_2_images
reset_folder('frames')
fps, i, interval = video_2_images(video_file, image_dir, image_file)
 
# display strat frame
from google.colab.patches import cv2_imshow
img = cv2.imread('frames/000000.jpg')
cv2_imshow(img)
 
# display parameter
print('fps = ', fps)
print('frames = ', i)
print('interval = ', interval)

In [ ]:
#@title **MediaPipe from frames to images**

import cv2
from google.colab.patches import cv2_imshow
import numpy as np
import glob
from tqdm import tqdm ###

# reset images folder
reset_folder('images')

# image file names to files in list format
files=[]
for name in sorted(glob.glob('./frames/*.jpg')):
    files.append(name)

# Read images with OpenCV.
images = {name: cv2.imread(name) for name in files}

#for name, image in images.items():
for name, image in tqdm(images.items()):  
  # Convert the BGR image to RGB and process it with MediaPipe Pose.
  results = holistic.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

  # Draw pose landmarks.
  annotated_image = image.copy()
  mp_drawing.draw_landmarks(
      image=annotated_image,
      landmark_list=results.left_hand_landmarks,
      connections=mp_holistic.HAND_CONNECTIONS,
      landmark_drawing_spec=drawing_dot_spec,
      connection_drawing_spec=drawing_hand_spec)
  mp_drawing.draw_landmarks(
      image=annotated_image,
      landmark_list=results.right_hand_landmarks,
      connections=mp_holistic.HAND_CONNECTIONS,
      landmark_drawing_spec=drawing_dot_spec,
      connection_drawing_spec=drawing_hand_spec)
  mp_drawing.draw_landmarks(
      image=annotated_image, 
      landmark_list=results.face_landmarks, 
      connections=mp_holistic.FACEMESH_TESSELATION,  ###
      landmark_drawing_spec=drawing_face_spec,
      connection_drawing_spec=drawing_face_spec)
  mp_drawing.draw_landmarks(
      image=annotated_image, 
      landmark_list=results.pose_landmarks, 
      connections=mp_holistic.POSE_CONNECTIONS,
      landmark_drawing_spec=drawing_pose_spec,
      connection_drawing_spec=drawing_pose_spec)
  
  save_name = 'images/'+os.path.basename(name) ###
  cv2.imwrite(save_name, annotated_image)  

In [ ]:
#@title **make movie from images**
#@markdown　check with_sound if the video has sound
with_sound = False #@param {type:"boolean"}
fps_r = fps/interval

print('making movie...')
if with_sound ==  True:  
  ! ffmpeg -y -r $fps_r -i images/%6d.jpg -vcodec libx264 -pix_fmt yuv420p -loglevel error out.mp4
  # audio extraction/addition
  print('preparation for sound...')
  ! ffmpeg -y -i $video_file -loglevel error sound.mp3
  ! ffmpeg -y -i out.mp4 -i sound.mp3 -loglevel error output.mp4
else:
  ! ffmpeg -y -r $fps_r -i images/%6d.jpg -vcodec libx264 -pix_fmt yuv420p -loglevel error output.mp4

display_mp4('output.mp4')

In [ ]:
#@title **download movie** (chrome)
from google.colab import files
files.download('output.mp4')